In [95]:
from pytrips.ontology import get_ontology as tripsont
from pytrips.structures import TripsType
from nltk.corpus.reader.wordnet import Synset
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
from collections import defaultdict
from collections import namedtuple
import random

from scipy.stats import spearmanr, pearsonr

nouns = list(wn.all_synsets(pos=wn.NOUN))
num = 100
noun_sample = random.sample(nouns, num)

In [2]:
"""
Default node weights, can be overrided for variations
"""
_node_weights = defaultdict(lambda: 1)

_node_weights["fakeroot"] = 30

def _fallback(x, func, nonzero=False):
    """
    Basic null checks for selecting a value from a list
    """
    if (not x) or (sum(x) == 0 and nonzero):
        return -1
    return func(x)

"""
Pick a value from a list
"""
ListStrategy = {
    'min': lambda x: _fallback(x, min),
    'max': lambda x: _fallback(x, max),
    'average': lambda x: _fallback(x, lambda l: sum(l)/len(l), nonzero=True)
}


"""
Find the last common element between two lists
"""
def last_overlap(v1, v2, aligned=None):
    if not v1 or not v2:
        return aligned
    elif v1[0] != v2[0]:
        return aligned
    else:
        return last_overlap(v1[1:], v2[1:], aligned=v1[0])

class SemNode:
    def __init__(self, node):
        self._node = node
    
    def __eq__(self, other):
        if issubclass(type(other), SemNode):
            if type(other.content) == type(self.content):
                return other.content == self.content
        return False
    
    def __hash__(self):
        return hash(self.__repr__())
    
    def __repr__(self):
        return "<SemNode: {}>".format(self.content.__repr__())
    
    def __str__(self):
        return "<SemNode: {}>".format(self.name)
    
    @property
    def name(self):
        """
        return canonical name for node
        """
        return str(self.content)
    
    @property
    def adjacent(self, label=None):
        """
        Return connected elements by label.  Useful for following non-hypernym relations in wordnet
        Can be used to implement other connections.
        Override as necessary.
        """
        return []
    
    @property
    def root(self):
        """
        Check if the node is a root of some sort.  Override as necessary for different resources.
        """
        return not self.parents
    
    @property
    def parents(self):
        """
        Get all parents for a node.  Perform any cross-resource cutoffs.
        """
        pass
    
    @property
    def children(self):
        """
        Get all children for a node.  Perform any cross resource cutoffs.
        """
        pass
    
    @property
    def content(self):
        """
        Return the wrapped node.
        """
        return self._node
    
    @property
    def resource(self):
        """
        Return the name of the relevant resource
        """
        return "default"
    
    def weight(self, weights=None):
        """
        Get resource-based weight of node.  Pass a weight dictionary as necessary.
        TODO: add an argument to pass individual weights
        """
        if not weights:
            weights = _node_weights
        return weights[self.resource]
    
    @staticmethod
    def path_depth(path, weights=None):
        """
        Get the total depth of a path, defined as a list of nodes.
        Does not check validity of path
        """
        return sum([p.weight(weights=weights) for p in path])
    
    @staticmethod
    def depth(node, weights=None, strategy='min'):
        """
        Get the depth of a node to a root using a weight dictionary and selection strategy.
        Default is minimum depth from any root.
        """
        weighted = [SemNode.path_depth(p, weights=weights) for p in node.paths_to_root()]
        return ListStrategy[strategy](weighted)
    
    @staticmethod
    def make(node):
        """
        Make a node based on the input type.
        Should add parameter dictionary to pass on to children
        """
        if type(node) is TripsType:
            return TripsNode(node)
        elif type(node) is Synset:
            return WordNetNode(node)
        elif type(node) is str:
            return WordNode(node)
        elif type(node) is SemNode:
            return node
        else:
            return None
    
    def paths_to_root(self):
        """
        Find all paths to a root based on hierarchy rules.
        Some resources return only one (Trips), others may return multiple (WordNet)
        """
        if self.root:
            return [[self]]
        res = []
        for c in self.parents:
            ptrs = c.paths_to_root()
            res.extend([t + [self] for t in ptrs if self not in t])
        return res
    
    def lcs_set(self, other):
        """
        Find the set of Lowest Common Subsumers for a node.  
        Some resources have only one (Trips) other can have multiple (WordNet)
        """
        lcs = [last_overlap(p,q) for p in self.paths_to_root() for q in other.paths_to_root()]
        filtered = [x for x in lcs if x]
        if not filtered:
            return [TripsNode(tripsont()["root"])]
        return filtered
    
    def wupalmer(self, other, weights=None, depth_strategy='min', lcs_strategy='max'):
        """
        return cross-wupalmer measure using provided weights, depth_strategy and lcs_strategy
        depth_strategy: Choose max, min, or average depth over all paths
        lcs_strategy: Choose max, min, or average depth of lcs over all alternatives
        """
        if not issubclass(type(other), SemNode):
            other = SemNode.make(other) # this would break passing in an arbitrary maker object
        lcs_depth = ListStrategy[lcs_strategy]([SemNode.depth(d, weights, depth_strategy) for d in self.lcs_set(other)])
        sd = SemNode.depth(self, weights, depth_strategy)
        od = SemNode.depth(other, weights, depth_strategy)
        return 2*lcs_depth/(sd + od)
    
    def path_similarity(self, other, weights=None, depth_strategy='min', lcs_strategy='min'):
        """
        Like wupalmer, except (d(s1) + d(s2) - 2 * lcs(s1,s2))
        """
        if not issubclass(type(other), SemNode):
            other = SemNode.make(other) # this would break passing in an arbitrary maker object
        lcs_depth = ListStrategy[lcs_strategy]([SemNode.depth(d, weights, depth_strategy) for d in self.lcs_set(other)])
        sd = SemNode.depth(self, weights, depth_strategy)
        od = SemNode.depth(other, weights, depth_strategy)
        return 1/(2 + sd + od - 2*lcs_depth)

In [3]:
class WordNode(SemNode):
    """
    Take a "word.pos" element as a node in the generalized hierarchy.
    """
    def resource(self):
        return "word"
    
    @property
    def name(self):
        return self.content
    
    @property
    def children(self):
        return []
    
    def word_pos(self):
        if "." in self._node:
            return self._node.split(".")
        return self._node, None
    
    @property
    def parents(self):
        """
        Lookup all TripsTypes, lookup all Wordnet Types.
        """
        w, p = self.word_pos()
        wordnet = wn.synsets(w, p)
        trips = tripsont().get_word(w, p)
        return [SemNode(c) for c in wordnet+trips]

class FakeRoot(SemNode):
    """
    FakeRoot for completeness purposes
    """
    def __init__(self):
        super(FakeRoot, self).__init__("fakeroot")
        
    def resource(self):
        return "fakeroot"
    
    @property
    def name(self):
        return "fakeroot"
    
    @property
    def parents(self):
        return []
    
    @property
    def children(self):
        return []
        

class TripsNode(SemNode):
    @property
    def name(self):
        return self.content.name
    
    @property
    def parents(self):
        return [SemNode.make(self._node.parent)]

    @property
    def children(self):
        return [SemNode.make(c) for c in self._node.children] + [SemNode.make(c) for c in self._node.wordnet]
    
    @property
    def resource(self):
        return "trips"
    
    @property
    def root(self):
        return self._node.depth == 0

    
class WordNetNode(SemNode):
    @property
    def name(self):
        return self.content.name()
    
    @property
    def parents(self):  
        # NOTE: actually this is a little bit of a problem because we're not taking
        #       WN hypernyms
        tt = tripsont()[self._node]
        if not tt:
            tt = self._node.hypernyms()
        return [SemNode.make(p) for p in tt]

    @property
    def children(self):
        return [SemNode.make(c) for c in self._node.hyponyms()]
    
    @property
    def resource(self):
        return "wordnet"
    

# tests

In [4]:
# equality

cat1 = wn.synset("cat.v.1")
cat2 = wn.synset("cat.v.1")

wcat1 = WordNetNode(cat1)
wcat2 = WordNetNode(cat2)

assert issubclass(type(wcat1), SemNode)
assert wcat1 == wcat2

# hypernyms

animal = SemNode.make(tripsont()["nonhuman-animal"])
mammal = SemNode.make(tripsont()["mammal"])
assert [mammal] == animal.parents


abbess = SemNode.make(wn.synset("abbess.n.1"))
scand  = SemNode.make(wn.synset("scandinavia.n.2"))
print(abbess.paths_to_root())
print(scand.paths_to_root())

wn.synset("scandinavia.n.2").hypernyms()

2019-05-01 01:34:56,667 : pytrips : INFO : Loading ontology
2019-05-01 01:34:56,669 : root : INFO : Getting ontology.json - version 0.1.15
2019-05-01 01:34:56,785 : pytrips : INFO : Loaded ontology
2019-05-01 01:34:56,789 : pytrips : INFO : Loading lexicon
2019-05-01 01:34:56,795 : root : INFO : Getting words.json - version 0.1.15
2019-05-01 01:34:57,096 : pytrips : INFO : Loaded lexicon
2019-05-01 01:34:57,098 : pytrips : INFO : AREA1767 has 2 senses
2019-05-01 01:34:57,100 : pytrips : INFO : MAYBE17874 has 2 senses
2019-05-01 01:34:57,101 : pytrips : INFO : DEMONSTRATIONS8016 has 6 senses
2019-05-01 01:34:57,102 : pytrips : INFO : PAN20471 has 2 senses
2019-05-01 01:34:57,103 : pytrips : INFO : CALM4270 has 2 senses
2019-05-01 01:34:57,110 : pytrips : INFO : CALM4264 has 2 senses
2019-05-01 01:34:57,118 : pytrips : INFO : ACTIVATE608 has 2 senses
2019-05-01 01:34:57,121 : pytrips : INFO : DEENERGISE7794 has 2 senses
2019-05-01 01:34:57,123 : pytrips : INFO : IGNITE14602 has 2 senses


2019-05-01 01:34:57,554 : pytrips : INFO : OIL19817 has 2 senses
2019-05-01 01:34:57,556 : pytrips : INFO : MENTION18094 has 3 senses
2019-05-01 01:34:57,558 : pytrips : INFO : GALVANIZE12445 has 2 senses
2019-05-01 01:34:57,559 : pytrips : INFO : SURRENDER28918 has 2 senses
2019-05-01 01:34:57,560 : pytrips : INFO : BAR2619 has 2 senses
2019-05-01 01:34:57,561 : pytrips : INFO : DEEPEN7814 has 2 senses
2019-05-01 01:34:57,562 : pytrips : INFO : DRY9508 has 2 senses
2019-05-01 01:34:57,563 : pytrips : INFO : DRAW9372 has 2 senses
2019-05-01 01:34:57,564 : pytrips : INFO : ROTATE24940 has 3 senses
2019-05-01 01:34:57,565 : pytrips : INFO : SMOOTH26990 has 2 senses
2019-05-01 01:34:57,565 : pytrips : INFO : RESULT24489 has 3 senses
2019-05-01 01:34:57,567 : pytrips : INFO : LEAN16673 has 3 senses
2019-05-01 01:34:57,570 : pytrips : INFO : ASSESS1992 has 2 senses
2019-05-01 01:34:57,572 : pytrips : INFO : HANDLE13498 has 2 senses
2019-05-01 01:34:57,575 : pytrips : INFO : DIVIDE9101 has 5

2019-05-01 01:34:57,830 : pytrips : INFO : HINDER13999 has 2 senses
2019-05-01 01:34:57,834 : pytrips : INFO : DERIDE8166 has 2 senses
2019-05-01 01:34:57,835 : pytrips : INFO : SENSE25839 has 2 senses
2019-05-01 01:34:57,837 : pytrips : INFO : SENSE25833 has 2 senses
2019-05-01 01:34:57,838 : pytrips : INFO : TAKE29266 has 10 senses
2019-05-01 01:34:57,840 : pytrips : INFO : MAD17518 has 3 senses
2019-05-01 01:34:57,841 : pytrips : INFO : UNDO31349 has 2 senses
2019-05-01 01:34:57,842 : pytrips : INFO : ENTHUSIASTIC10278 has 3 senses
2019-05-01 01:34:57,843 : pytrips : INFO : LAMBASTE16460 has 2 senses
2019-05-01 01:34:57,844 : pytrips : INFO : DEACTIVATE7657 has 2 senses
2019-05-01 01:34:57,844 : pytrips : INFO : AGENDA905 has 2 senses
2019-05-01 01:34:57,845 : pytrips : INFO : DIMENSION8536 has 2 senses
2019-05-01 01:34:57,847 : pytrips : INFO : FLOAT11750 has 2 senses
2019-05-01 01:34:57,848 : pytrips : INFO : SUBSTITUTIONS28644 has 2 senses
2019-05-01 01:34:57,849 : pytrips : INFO

2019-05-01 01:34:58,099 : pytrips : INFO : DE-PUNC-MINUS-ENERGISE7633 has 2 senses
2019-05-01 01:34:58,100 : pytrips : INFO : STRIP28446 has 2 senses
2019-05-01 01:34:58,102 : pytrips : INFO : TRANSFORM30617 has 2 senses
2019-05-01 01:34:58,102 : pytrips : INFO : SWAP28982 has 2 senses
2019-05-01 01:34:58,104 : pytrips : INFO : PAIN20410 has 2 senses
2019-05-01 01:34:58,106 : pytrips : INFO : SHOUT26301 has 3 senses
2019-05-01 01:34:58,107 : pytrips : INFO : SHOUTS26303 has 3 senses
2019-05-01 01:34:58,109 : pytrips : INFO : STANDARD27995 has 2 senses
2019-05-01 01:34:58,110 : pytrips : INFO : REFUTE23843 has 3 senses
2019-05-01 01:34:58,111 : pytrips : INFO : PAPER20521 has 2 senses
2019-05-01 01:34:58,112 : pytrips : INFO : PAPER20518 has 2 senses
2019-05-01 01:34:58,112 : pytrips : INFO : RETRACT24523 has 2 senses
2019-05-01 01:34:58,114 : pytrips : INFO : BLOAT3280 has 2 senses
2019-05-01 01:34:58,115 : pytrips : INFO : BLOAT3274 has 2 senses
2019-05-01 01:34:58,115 : pytrips : INF

2019-05-01 01:34:58,261 : pytrips : INFO : FLOWER11812 has 2 senses
2019-05-01 01:34:58,262 : pytrips : INFO : DETECT8295 has 2 senses
2019-05-01 01:34:58,263 : pytrips : INFO : LIGHT16959 has 3 senses
2019-05-01 01:34:58,264 : pytrips : INFO : DISCOURAGE8760 has 3 senses
2019-05-01 01:34:58,265 : pytrips : INFO : PASS20645 has 3 senses
2019-05-01 01:34:58,266 : pytrips : INFO : DISAGREE8641 has 3 senses
2019-05-01 01:34:58,267 : pytrips : INFO : CRUMMY7270 has 4 senses
2019-05-01 01:34:58,268 : pytrips : INFO : MOROSE18664 has 3 senses
2019-05-01 01:34:58,268 : pytrips : INFO : RECHARGE23612 has 2 senses
2019-05-01 01:34:58,269 : pytrips : INFO : PURIFY23023 has 3 senses
2019-05-01 01:34:58,270 : pytrips : INFO : WORRY33206 has 3 senses
2019-05-01 01:34:58,271 : pytrips : INFO : SHOUT26301 has 3 senses
2019-05-01 01:34:58,272 : pytrips : INFO : SIMMER26530 has 2 senses
2019-05-01 01:34:58,273 : pytrips : INFO : SURVIVE28937 has 2 senses
2019-05-01 01:34:58,274 : pytrips : INFO : STIMU

...

2019-05-01 01:35:03,601 : pytrips : INFO : TILT30133 has 2 senses
2019-05-01 01:35:03,602 : pytrips : INFO : THEN29811 has 2 senses
2019-05-01 01:35:03,603 : pytrips : INFO : DRIFT9416 has 2 senses
2019-05-01 01:35:03,605 : pytrips : INFO : NOTICE19475 has 3 senses
2019-05-01 01:35:03,606 : pytrips : INFO : RECHARGE23612 has 2 senses
2019-05-01 01:35:03,607 : pytrips : INFO : COMBINE5899 has 2 senses
2019-05-01 01:35:03,608 : pytrips : INFO : ASSERT1983 has 2 senses
2019-05-01 01:35:03,609 : pytrips : INFO : WILT32978 has 2 senses
2019-05-01 01:35:03,610 : pytrips : INFO : GRUMPY13295 has 3 senses
2019-05-01 01:35:03,611 : pytrips : INFO : REPORT24196 has 4 senses
2019-05-01 01:35:03,611 : pytrips : INFO : ARRANGEMENT1822 has 2 senses
2019-05-01 01:35:03,612 : pytrips : INFO : COMBINATION5907 has 2 senses
2019-05-01 01:35:03,613 : pytrips : INFO : ACCUMULATE499 has 2 senses
2019-05-01 01:35:03,613 : pytrips : INFO : DEPARTURE8076 has 2 senses
2019-05-01 01:35:03,614 : pytrips : INFO : 

2019-05-01 01:35:03,767 : pytrips : INFO : COMMEND5964 has 2 senses
2019-05-01 01:35:03,768 : pytrips : INFO : ABHOR313 has 2 senses
2019-05-01 01:35:03,768 : pytrips : INFO : SHOW26321 has 7 senses
2019-05-01 01:35:03,769 : pytrips : INFO : ESCAPE10402 has 2 senses
2019-05-01 01:35:03,770 : pytrips : INFO : ABSTRACT386 has 2 senses
2019-05-01 01:35:03,771 : pytrips : INFO : COORDINATE6780 has 2 senses
2019-05-01 01:35:03,772 : pytrips : INFO : START28047 has 2 senses
2019-05-01 01:35:03,773 : pytrips : INFO : START28042 has 2 senses
2019-05-01 01:35:03,774 : pytrips : INFO : START28028 has 6 senses
2019-05-01 01:35:03,775 : pytrips : INFO : EFFECTIVE9776 has 4 senses
2019-05-01 01:35:03,776 : pytrips : INFO : APPEAR1597 has 6 senses
2019-05-01 01:35:03,777 : pytrips : INFO : COMPANY6025 has 2 senses
2019-05-01 01:35:03,778 : pytrips : INFO : HERALD13872 has 2 senses
2019-05-01 01:35:03,778 : pytrips : INFO : COINCIDE5793 has 2 senses
2019-05-01 01:35:03,779 : pytrips : INFO : RIDICULE

2019-05-01 01:35:03,883 : pytrips : INFO : EVEN10525 has 3 senses
2019-05-01 01:35:03,884 : pytrips : INFO : RUE25033 has 2 senses
2019-05-01 01:35:03,884 : pytrips : INFO : TRAVEL30721 has 3 senses
2019-05-01 01:35:03,885 : pytrips : INFO : LINK17084 has 3 senses
2019-05-01 01:35:03,886 : pytrips : INFO : PREVENT22394 has 2 senses
2019-05-01 01:35:03,887 : pytrips : INFO : INTERVENE15563 has 2 senses
2019-05-01 01:35:03,887 : pytrips : INFO : LOVE17375 has 2 senses
2019-05-01 01:35:03,889 : pytrips : INFO : LOOK-LIKE17304 has 2 senses
2019-05-01 01:35:03,890 : pytrips : INFO : LOOK17295 has 4 senses
2019-05-01 01:35:03,890 : pytrips : INFO : LOOK17290 has 2 senses
2019-05-01 01:35:03,891 : pytrips : INFO : GRANT13077 has 2 senses
2019-05-01 01:35:03,892 : pytrips : INFO : SPEAK27536 has 4 senses
2019-05-01 01:35:03,893 : pytrips : INFO : VALUE31959 has 2 senses
2019-05-01 01:35:03,894 : pytrips : INFO : VALUE31953 has 2 senses
2019-05-01 01:35:03,895 : pytrips : INFO : SOLUTION27288 h

[[<SemNode: ont::root>, <SemNode: ont::any-sem>, <SemNode: ont::referential-sem>, <SemNode: ont::phys-object>, <SemNode: ont::natural-object>, <SemNode: ont::organism>, <SemNode: ont::animal>, <SemNode: ont::vertebrate>, <SemNode: ont::mammal>, <SemNode: ont::person>, <SemNode: Synset('abbess.n.01')>]]
[[<SemNode: Synset('scandinavia.n.02')>]]


[]

# Evaluation tools

from collections import namedtuple, Counter
from tqdm import tnrange, tqdm_notebook

Comp = namedtuple("comp", ["n1", "n2", "normal", "cross"])
def comp_string(c):
    return """
    node1: {}
    node1: {}
    wordnet only wup: {}
    cross wupalmer:   {}
    """.format(c.n1, c.n2, c.normal, c.cross)

def compare_wup(n1, n2):
    cross = SemNode.make(n1).wupalmer(SemNode.make(n2))
    normal= n1.wup_similarity(n2)
    return Comp(n1, n2, normal, cross)

res = []
for i in tqdm_notebook(range(num), desc="i"):
    for j in range(i, num):
        res.append(compare_wup(noun_sample[i], noun_sample[j]))

# count data
ranges_normal = Counter()
ranges_cross = Counter()
for i in res:
    ranges_normal[i.normal] += 1
    ranges_cross[i.cross] += 1

def binning(data, n=10):
    bin_width = 1/n
    binned = []
    for d in data:
        try:
            binned.append(int(n*d)/n)
        except:
            print(n, d, int(n*d))
    return binned

def bin_count(data, n=10):
    b_counter = Counter()
    bin_width = 1/n
    binned = []
    for d in data:
        try:
            b_counter[int(n*d)/n] += 1
        except:
            print(n, d, int(n*d))
    return b_counter

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

bins=25
data = binning([i.normal for i in res], bins)

counted = sorted(bin_count([i.normal for i in res], n=bins).items(), key=lambda x: x[0])
for c in counted:
    print(c)


plt.hist(data, density=True, bins=bins)
plt.ylabel('WordnetOnly');

data = binning([i.cross for i in res if i.cross <= 1], bins)

counted = sorted(bin_count([i.cross for i in res], n=bins).items(), key=lambda x: x[0])
for c in counted:
    print(c)

for i in res:
    if 0 > i.cross or 1 < i.cross:
        print(i)

plt.hist(data, density=True, bins=bins)
plt.ylabel('Trips-WUPalmer');

In [5]:
import pandas as pd

simlex = pd.read_csv("../SimLex-999/SimLex-999.txt", sep="\t")

In [126]:
def pops(l):
    if l:
        return l[0]
    return None

def collects(l):
    return [y for y in l if y]

def n_or_v(synset):
    return synset.pos() in [wn.NOUN, wn.VERB]

def resnik(s1, s2, ic_corpus):
    if n_or_v(s1) and n_or_v(s2):
        return s1.res_similarity(s2, ic_corpus)
    return s1.wup_similarity(s2)

def _list_fallback(func, fallback_func, args):
    result = func(*args)
    if not result:
        result = fallback_func(*args)
    return result
    
sim_strategy = {
    "mfs": lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
    "average": lambda x, p: [SemNode.make(s) for s in wn.synsets(x, p)],
    "word": lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)],
    "lookup": lambda x, p: _list_fallback(
        lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)], 
        lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
        [x, p]
    ),
}

sim_metric = {
    "cross" : lambda x, y: x.wupalmer(y),
    "tripspath" : lambda x, y: x.path_similarity(y),
    "normal": lambda x, y: x.content.wup_similarity(y.content),
    "resnik_brown": lambda x, y: resnik(x.content, y.content, brown_ic),
    "resnik_semcor": lambda x, y: resnik(x.content, y.content, semcor_ic)
}

def similarity_test(word1, word2, pos=None, metric="cross", strategy="average", select1="max", select2="max"):
    if not pos:
        pos = "nvar"
    else:
        pos = pos.lower()[0]
    metric = sim_metric[metric]
    strategy = sim_strategy[strategy]
    results = []
    for x in pos:
        word1_node = collects(strategy(word1, x))
        word2_node = collects(strategy(word2, x))
        if word1_node and word2_node:
            scores = collects([metric(x, y) for x in word1_node for y in word2_node])
            results.append(ListStrategy[select1](scores))
    if results:
        return ListStrategy[select2](results)
    return -1
    

In [127]:
ws353 = []
cross = []
regular=[]

def get_valid_scores(l1, l2):
    res1 = []
    res2 = []
    for x, y in zip(l1, l2):
        if y >= 0:
            res1.append(x)
            res2.append(y)
    return res1, res2 

In [128]:
SimExperiment = namedtuple("SimExperiment", ["name", "data", "metric", "strategy", "select1", "select2"])
SimTask = namedtuple("SimTask", ["word1", "word2", "gold", "pos"])
SimTaskResults = namedtuple("SimTaskResults", ["experiment", "instances", "spearman","pearson"])

def ws353(name, metric, strategy, select1="max", select2="max"):
    wordsim2 = pd.read_csv("../wordsim353/set2.tab", sep="\t")
    wordsim1= pd.read_csv("../wordsim353/set1.tab", sep="\t")
    res = []
    for i, row in wordsim1.iterrows():
        res.append(SimTask(row[0], row[1], row[2], None))
    for i, row in wordsim2.iterrows():
        res.append(SimTask(row[0], row[1], row[2], None))
    return SimExperiment(name, res, metric, strategy, select1, select2)

def run_experiment(exp):
    results = []
    for d in exp.data:
        results.append(
            similarity_test(d.word1, d.word2, 
                            metric=exp.metric, 
                            strategy=exp.strategy, 
                            select1=exp.select1, select2=exp.select2)
        )
    
    results, gold = get_valid_scores([d.gold for d in exp.data], results)
    return SimTaskResults(exp, results, spearmanr(results, gold), pearsonr(results, gold))

def experiment_string(exp, pandas=True, dataframe=None):
    columns = "name metric candidate choice1 choice2 instances spearmanr spearmanp pearsonr pearsonp".split()
    if dataframe is None:
        dataframe = pd.DataFrame(columns=columns)
    e = exp.experiment
    if pandas:
        values = [e.name, e.metric, e.strategy, e.select1, e.select2, len(exp.instances),
        exp.spearman.correlation, exp.spearman.pvalue, 
        exp.pearson[0], exp.pearson[1]]
        res = pd.DataFrame({v: [r] for r, v in zip(values, columns)})
        return dataframe.append(res)
    return """
    ---
    name:      {}
    metric:    {}
    candidate: {}
    choice:    {}/{}
    --- 
    instances:    {}
    spearman rho: {}
    p-value:      {}
    
    pearson rho:  {}
    p-value:      {}
    ================
    """.format(
        e.name, e.metric, e.strategy, e.select1, e.select2, len(exp.instances),
        exp.spearman.correlation, exp.spearman.pvalue, 
        exp.pearson[0], exp.pearson[1]
    )
    

In [132]:
df = experiment_string(
        run_experiment(ws353("base", "normal", "mfs")))

df = experiment_string(
        run_experiment(ws353("base", "normal", "average")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("base", "resnik_brown", "mfs")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("base", "resnik_brown", "average")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("base", "resnik_semcor", "mfs")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("base", "resnik_semcor", "average")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("base", "resnik_semcor", "lookup")), dataframe=df)

df

/Users/rik/.pyenv/versions/3.5.5/envs/nlp/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/rik/.pyenv/versions/3.5.5/envs/nlp/lib/python3.5/site-packages/scipy/stats/stats.py:5833: RuntimeWarning: overflow encountered in multiply
  return np.sum(a*a, axis)


AttributeError: 'TripsType' object has no attribute 'pos'

In [133]:
# Tripswordnet

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "cross", "mfs")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "cross", "average")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "cross", "word")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "cross", "lookup")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "tripspath", "mfs")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "tripspath", "average")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "tripspath", "word")), dataframe=df)

df = experiment_string(
        run_experiment(ws353("Trips-Wordnet", "tripspath", "lookup")), dataframe=df)

In [134]:
df

,candidate,choice1,choice2,instances,metric,name,pearsonp,pearsonr,spearmanp,spearmanr
0,mfs,max,max,351,normal,base,4.177583e-09,0.307171,3.123276e-07,0.268975
0,average,max,max,351,normal,base,2.103189e-09,0.312763,1.009547e-11,0.352748
0,mfs,max,max,260,resnik_brown,base,1.486401e-11,0.402657,9.384770e-10,0.367897
0,average,max,max,325,resnik_brown,base,3.351719e-13,0.389255,8.488894e-12,0.366977
0,mfs,max,max,260,resnik_semcor,base,1.751398e-11,0.401356,4.722929e-11,0.393365
0,average,max,max,325,resnik_semcor,base,1.000000e+00,0.000000,7.702590e-12,0.367672
0,mfs,max,max,351,cross,Trips-Wordnet,2.215061e-04,0.195910,3.440028e-04,0.190004
0,average,max,max,351,cross,Trips-Wordnet,1.864527e-04,0.198175,2.642768e-06,0.247663
0,word,max,max,193,cross,Trips-Wordnet,5.135443e-09,0.405036,1.933082e-07,0.364133
0,lookup,max,max,351,cross,Trips-Wordnet,9.526882e-06,0.233879,2.027020e-05,0.225354
